In [2]:
#Import all the packages And always Check that the version of the Chromedriver is up to date
#https://sites.google.com/a/chromium.org/chromedriver/downloads

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import requests
import datetime
import os
import PyPDF2
import re

#Run all the cells in order to obtain the best results!


In [2]:
# Open Chrome Driver
path= "C:\Program Files (x86)\chromedriver"
driver = webdriver.Chrome(path)

In [3]:
#Go to the Main Page
url="https://app.vts.com/users/sign_in"
driver.get(url)

In [4]:
#Login into VTS
user=driver.find_element_by_xpath("//input[@id='user_email']")
password=driver.find_element_by_xpath("//input[@id='user_password']")
user.send_keys('XXXXXXXXXXXXXX')
password.send_keys('XXXXXXXXXXXXX')
password.send_keys(Keys.ENTER)
driver.implicitly_wait(5)

In [5]:
# Load VTS Properties and get URLS
vts_properties = pd.read_excel ('VTS_Property_list.xlsx')
URLS=vts_properties['Link_Report'].tolist()
NAMES=vts_properties['Property'].tolist()

In [6]:
#Helper Function 1: Functions that checks if a directory changes. necesary 
def download_wait(path_to_downloads):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < 15:
        time.sleep(1)
        dl_wait = False
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.pdf'):
                dl_wait = True
        seconds += 1
    return seconds

In [4]:
#Helper Function 2: Creates a list of dates

def gen_dates(start_date,end_date):
    List_dates=[]
    start = datetime.datetime.strptime(start_date, "%m/%d/%y")
    end = datetime.datetime.strptime(end_date, "%m/%d/%y")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    for date in date_generated:
        List_dates.append(date.strftime("%m/%d/%y"))
    return List_dates   


In [8]:
#Hardcoded Variables
List_URLS=vts_properties['Link_Report'].tolist()
path_to_downloads='C:/Users/reinaldo.viccini/Downloads'
List_dates=[]
List_names=vts_properties['Property'].tolist()

#Download all the reports
for url in List_URLS[0:]:
    #Go to the Report URL
    driver.get(url)
    #Export the report
    report=driver.find_element_by_xpath("//button[contains(@title,'Export Asset profile Report')]")
    report.click()
    driver.implicitly_wait(5)
    export=driver.find_element_by_xpath("(//span[@class='button-text'][contains(.,'Export')])[9]")
    export.click()
    #Wait until the download is done.
    download_wait(path_to_downloads)
print('Finished!')

      


In [6]:
#Make sure you see the files in downloads folders. And be careful with the names!!!


start_date="02/02/21"
end_date="02/18/21"
date_of_reports="property_overview-02-18-2021"
path_to_downloads='C:/Users/reinaldo.viccini/Downloads'
list_files = []


#List of files that are reports
for i in os.listdir(path_to_downloads):
    if date_of_reports in i:
        list_files.append(path_to_downloads+'/'+i)
        
#List of dates of interest:
List_dates=gen_dates(start_date,end_date)



In [7]:
# CHeck names, of the files and clean them

List_File_Name=[]

#extract Property names from the PDFs
for report in list_files:
    pdfFileObj = open(report, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pageObj = pdfReader.getPage(0)
    List_File_Name.append(pageObj.extractText())
    pdfFileObj.close()

List_File_Clean=[]    
for index, i in enumerate(List_File_Name):
    split_string = i.split("\n", 1)
    split_string=split_string[0]
    split_string = split_string.split("Calgary", 1)
    split_string=split_string[0]
    split_string = split_string.split("Southeast", 1)
    split_string=split_string[0]
    split_string = split_string.split("Northwest", 1)
    split_string=split_string[0]
    split_string = split_string.split("Northeast", 1)
    split_string=split_string[0]
    split_string = split_string.split("Southwest", 1)
    split_string=split_string[0]
    List_File_Clean.append(split_string)
    #print(str(index)+'  '+str(split_string))
  

In [8]:
#Open all PDF files and change the file name if the are files within the date range onlye print files with a match
for name,report in zip(List_File_Clean,list_files):
    pdfFileObj = open(report, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    n_pages = pdfReader.getNumPages()
    List_PDF=[]
    NEW_FILE_NAME=name
    for page in range(n_pages):
        pageObj = pdfReader.getPage(page)
        List_PDF.append(pageObj.extractText())
    print(str(index) +"len_PDF "+str(len(List_PDF))+ ' N_pages: ' +str(n_pages))   
    for date in List_dates:
        if any(date in mystring for mystring in List_PDF):
            newFile = open(str(name) +'.pdf', 'wb')
            pdfWriter = PyPDF2.PdfFileWriter()
            for page_original in range(n_pages):
                PDF_PAGE =pdfReader.getPage(page_original) 
                pdfWriter.addPage(PDF_PAGE) 
            pdfWriter.write(newFile)
            newFile.close()
            print("Match! :" + str(name)+' ' + str(date))
            break
        


65len_PDF 6 N_pages: 6
65len_PDF 3 N_pages: 3
65len_PDF 7 N_pages: 7
65len_PDF 22 N_pages: 22
Match!340 - 12thAvenue SW02/03/21
65len_PDF 3 N_pages: 3
65len_PDF 9 N_pages: 9
Match!3816 Bow Trail02/02/21
65len_PDF 6 N_pages: 6
Match!416 MeridianRoad 02/09/21
65len_PDF 6 N_pages: 6
65len_PDF 17 N_pages: 17
Match!4625 Varsity02/10/21
65len_PDF 14 N_pages: 14
Match!525 11 Avenue02/03/21
65len_PDF 6 N_pages: 6
65len_PDF 4 N_pages: 4
65len_PDF 11 N_pages: 11
Match!5819 2nd StreetSW02/11/21
65len_PDF 6 N_pages: 6
65len_PDF 5 N_pages: 5
Match!7110 44 Street02/08/21
65len_PDF 11 N_pages: 11
65len_PDF 2 N_pages: 2
65len_PDF 5 N_pages: 5
65len_PDF 3 N_pages: 3
65len_PDF 5 N_pages: 5
65len_PDF 9 N_pages: 9
Match!3553 31 StreetNW02/03/21
65len_PDF 9 N_pages: 9
Match!340 Midpark WaySE02/02/21
65len_PDF 9 N_pages: 9
Match!1302 4 Street SW02/03/21
65len_PDF 4 N_pages: 4
Match!5716 1 Street02/10/21
65len_PDF 1 N_pages: 1
65len_PDF 7 N_pages: 7
65len_PDF 15 N_pages: 15
Match!75 Crowfoot Rise02/03/21
65l